In [15]:

function inexact_method(f,g,ϕ0,ϕd0,xk,dk;τ=0.5,ϵ=0.5,ζ=2)
    α=1
    dα=dk'*g((xk.+α.*dk)...)
    while dα<0
        α=ζ*α
        dα=dk'*g((xk.+α.*dk)...)
    end
    ϕα=f((xk.+α.*dk)...)
    while(ϕα>ϕ0+ϵ*α*ϕd0)
        α=τ*α
        ϕα=f((xk.+α.*dk)...)
    end
    return α 
end


inexact_method (generic function with 1 method)

In [16]:

function gradient_descent(f,g,x0;
        ϵg=0.0001,ϵx=0.0001,ϵf=0.0001,maxIteractions=128,verbose=true)
    x1=x0
    d=-g(x1...)
    f1=f(x1...)
    ng=norm(d)
    α=inexact_method(f,g,f1,-ng'*ng,x1,d)
    x2=x1+α.*d
    f2=f(x2...)
    g2=g(x2...)
    nx=α*ng
    ni=1
    println("step:",ni,"   x:",x2,"   f:",f2,"  α=",α,"    g:",g2)
    while((ng>ϵg || nx>ϵx || abs(f2-f1)>ϵf )&& ni<maxIteractions )
        x1=x2
        d=-g(x1...)
        f1=f2
        ng=norm(d)
        α=inexact_method(f,g,f1,-ng'*ng,x1,d)
        x2=x1+α.*d
        f2=f(x2...)
        g2=g(x2...)
        nx=α*ng                  
        ni+=1
        if verbose
            println("step:",ni,"   x:",x2,"   f:",f2,"    g:",g2)
            println("   d:" ,d,"  α:",α)
        end
    end
    if ni>=maxIteractions
        println("warning:Iteractions exceeds",maxIteractions)
    end
    return x2,f2
end


gradient_descent (generic function with 1 method)

In [17]:

gradient_descent(
    (x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2 ; 200y-200x^2],
    [-2 2]', ϵg=0.0001,ϵx=0.0001,ϵf=0.0001,maxIteractions=128,verbose=false)


step:1   x:[-1.60791; 2.09766]   f:30.58816017707636  α=0.000244140625    g:[-318.899, -97.5438]


([0.765276; 0.585167], 0.05511859817478085)

In [18]:

function newton(f,g,h,x0,ϵ)
    xk=x0
    xk1=xk-inv(h(xk...))*g(xk...)
    i=1
    fx=f(xk1...)
        println("第",i,"次迭代")
    d=inv(h(xk...))*g(xk...)
    println("g=",g(xk...),"H=",h(xk...),"HT=",inv(h(xk...)),"d=",d)
        println("x=",xk1)
        println("fx=",fx)
    while(norm(xk1-xk)>=ϵ)
        i=i+1
        xk=xk1
        xk1=xk-inv(h(xk...))*g(xk...)
        fx=f(xk1...)
        println("第",i,"次迭代")
            d=inv(h(xk...))*g(xk...)
    println("g=",g(xk...),"H=",h(xk...),"HT=",inv(h(xk...)),"d=",d)
        println("x=",xk1)
        println("fx=",fx)
    end
end

newton (generic function with 1 method)

In [19]:
newton((x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2  200y-200x^2]',
    (x,y)->[1200x^2-400y+2 -400x;-400x 200],
    [-2 2]',
    0.0001 )

第1次迭代
g=[-1606; -400]H=[4002 800; 800 200]HT=[0.00124688 -0.00498753; -0.00498753 0.0249501]d=[-0.0074813; -1.97007]
x=[-1.99252; 3.97007]
fx=8.955168502514422
第2次迭代
g=[-6.02965; -0.011194]H=[3178.13 797.007; 797.007 200.0]HT=[0.494465 -1.97046; -1.97046 7.85736]d=[-2.95939; 11.7932]
x=[0.966873; -7.82315]
fx=7670.252979776174
第3次迭代
g=[3387.08; -1751.6]H=[4253.07 -386.749; -386.749 200.0]HT=[0.000285291 0.000551679; 0.000551679 0.00606681]d=[-1.89018e-5; -8.75803]
x=[0.966892; 0.934879]
fx=0.001096166665157438
第4次迭代
g=[-0.0662167; -7.14559e-8]H=[749.903 -386.757; -386.757 200.0]HT=[0.5 0.966892; 0.966892 1.87476]d=[-0.0331084; -0.0640245]
x=[1.0; 0.998904]
fx=0.00012015810143631642
第5次迭代
g=[0.438467; -0.219233]H=[802.438 -400.0; -400.0 200.0]HT=[0.410094 0.820188; 0.820188 1.64538]d=[-1.94038e-9; -0.00109617]
x=[1.0; 1.0]
fx=1.8096897961233417e-19
第6次迭代
g=[-8.50809e-10; 0.0]H=[802.0 -400.0; -400.0 200.0]HT=[0.5 1.0; 1.0 2.005]d=[-4.25404e-10; -8.50809e-10]
x=[1.0; 1.0]
fx=0.0


In [20]:
function levenberg_marquardt(f,g,h,x0,ϵ,u)
    xk=x0
    xk1=xk-inv(h(xk...)+u*I)*g(xk...)
    fx=f(xk1...)
    i=1
      println("第",i,"次迭代")
        println("x=",xk1)
        println("fx=",fx)
    while(norm(xk1-xk)>=ϵ)
        i=i+1
        xk=xk1
        xk1=xk-inv(h(xk...)+u*I)*g(xk...)
        fx=f(xk1...)
        println("第",i,"次迭代")
        println("x=",xk1)
        println("fx=",fx)
    end
end

levenberg_marquardt (generic function with 1 method)

In [21]:
levenberg_marquardt((x,y)->(1-x)^2+100(y-x^2)^2,
    (x,y)->[-400x*(y-x^2)+2*x-2  200y-200x^2]',
    (x,y)->[1200x^2-400y+2 -400x;-400x 200],
    [-2 2]',
    0.0001,10)

第1次迭代
x=[-1.91477; 3.58009]
fx=9.240124057467765
第2次迭代
x=[-1.86656; 3.48641]
fx=8.217736921258645
第3次迭代
x=[-1.82727; 3.34448]
fx=7.996551741507284
第4次迭代
x=[-1.7874; 3.20042]
fx=7.772760175905116
第5次迭代
x=[-1.74645; 3.05565]
fx=7.5460766132419534
第6次迭代
x=[-1.70431; 2.91018]
fx=7.3163304542099485
第7次迭代
x=[-1.66089; 2.76399]
fx=7.0832939636783845
第8次迭代
x=[-1.61608; 2.61704]
fx=6.846709526441185
第9次迭代
x=[-1.56974; 2.46931]
fx=6.6062858855573685
第10次迭代
x=[-1.52172; 2.32075]
fx=6.361692208064822
第11次迭代
x=[-1.47185; 2.17134]
fx=6.112550846380139
第12次迭代
x=[-1.41994; 2.02105]
fx=5.858428587677355
第13次迭代
x=[-1.36573; 1.86985]
fx=5.598826154095268
第14次迭代
x=[-1.30895; 1.71773]
fx=5.333165814035672
第15次迭代
x=[-1.24925; 1.5647]
fx=5.060777276707937
第16次迭代
x=[-1.18621; 1.41081]
fx=4.780882844115228
第17次迭代
x=[-1.11932; 1.25614]
fx=4.49258470663903
第18次迭代
x=[-1.04796; 1.10089]
fx=4.194861678784452
第19次迭代
x=[-0.971352; 0.94543]
fx=3.88659266832664
第20次迭代
x=[-0.888536; 0.790389]
fx=3.5666463961330144
第21次迭